In [8]:
%load_ext autoreload
%autoreload 2

# To load the environment variable defined in the .env file
from dotenv import load_dotenv
load_dotenv();

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'dotenv'

In [9]:
import src.data.NHLDataManager as DataManager

In [15]:
data_manager = DataManager.NHLDataManager(r'C:\Users\anniw\OneDrive - Universite de Montreal\MSc 2022-2024\IFT6758 Science des données\nhl_data_dir')

In [17]:
cd C:\Users\anniw\IFT6758-A2022-G08\notebooks

C:\Users\anniw\IFT6758-A2022-G08\notebooks


In [23]:
import os
import json
import pprint

import numpy as np

import ipywidgets as widgets
from ipywidgets import interact

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from matplotlib.ticker import FormatStrFormatter


@ interact(season_year=widgets.IntSlider(min=2016, max=2020, step=1, value=1, description='Year'), 
           season_type=widgets.Dropdown(options=data_manager.season_types, value=data_manager.season_types[0], description='Season'))
def interactive_tool(season_year: int, season_type: str):
      
    game_numbers = data_manager.get_game_numbers(season_year, season_type)

    @ interact(game_num=widgets.SelectionSlider(options=game_numbers, value=game_numbers[0], description='Game ID'))
    def load_json(game_num: int): 
                    
        data = data_manager.load_game(season_year, season_type, game_num)
                   
        # -- Header
        print(data['gameData']['datetime']['dateTime'])
        
        abb_home = data['gameData']['teams']['home']['abbreviation']
        abb_away = data['gameData']['teams']['away']['abbreviation']
        print(f'Game ID: {game_num}; {abb_home} (home) vs {abb_away} (away)\n')
        
        # -- Table
        print('{}{}{}'.format(''.ljust(15), 'Home'.ljust(7), 'Away'.ljust(7)))
        print('{}{}{}'.format('Teams:'.ljust(15), abb_home.ljust(7), abb_away.ljust(7)))
        
        goals_home = str(data['liveData']['linescore']['teams']['home']['goals'])
        goals_away = str(data['liveData']['linescore']['teams']['away']['goals'])
        print('{}{}{}'.format('Goals (line):'.ljust(15), goals_home.ljust(7), goals_away.ljust(7)))
        
        goals_home1 = str(data['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']['goals'])
        goals_away1 = str(data['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']['goals'])
        print('{}{}{}'.format('Goals (box):'.ljust(15), goals_home1.ljust(7), goals_away1.ljust(7)))

        sog_home = str(data['liveData']['linescore']['teams']['home']['shotsOnGoal'])
        sog_away = str(data['liveData']['linescore']['teams']['away']['shotsOnGoal'])
        print('{}{}{}'.format('SoG:'.ljust(15), sog_home.ljust(7), sog_away.ljust(7)))
         
        so_goals_home = str(data['liveData']['linescore']['shootoutInfo']['home']['scores'])
        so_goals_away = str(data['liveData']['linescore']['shootoutInfo']['away']['scores'])
        print('{}{}{}'.format('SO Goals:'.ljust(15), so_goals_home.ljust(7), so_goals_away.ljust(7)))
            
        so_attempts_home = str(data['liveData']['linescore']['shootoutInfo']['home']['attempts'])
        so_attempts_away = str(data['liveData']['linescore']['shootoutInfo']['away']['attempts'])
        print('{}{}{}'.format('SO Attempts:'.ljust(15), so_attempts_home.ljust(7), so_attempts_away.ljust(7)))
        
        @ interact(event_idx=widgets.IntSlider(min=0, max=max(0, len(data['liveData']['plays']['allPlays'])-1), step=1, value=0, description='Event Index'))
        def print_info(event_idx: int):
    
            plt.figure(figsize = (10, 5), dpi=100)
            img = mpimg.imread("./figures/nhl_rink.png")
            plt.imshow(img, extent=[-100.0, 100.0, -42.5, 42.5])
            
            plt.xlabel('feet')
            plt.ylabel('feet')
#             plt.yticks(np.linspace(-42.5, 42.5, 21.25))
            
            if data['gameData']['status']['abstractGameState'] != 'Preview':
                title_desc = data['liveData']['plays']['allPlays'][event_idx]['result']['description'] 
                title_period_time = data['liveData']['plays']['allPlays'][event_idx]['about']['periodTime']
                title_period = data['liveData']['plays']['allPlays'][event_idx]['about']['period']
                plt.suptitle(f'{title_desc}\n{title_period_time} P-{title_period}')
            else:
                pass
            
            title_away_abb = data['gameData']['teams']['away']['abbreviation']
            title_home_abb = data['gameData']['teams']['home']['abbreviation']
            plt.title('{}{}'.format(title_away_abb.center(60), title_home_abb.center(60)), horizontalalignment='center')
              
            try: # plot event if exists
                x_coord = data['liveData']['plays']['allPlays'][event_idx]['coordinates']['x']
                y_coord = data['liveData']['plays']['allPlays'][event_idx]['coordinates']['y']
                plt.plot(x_coord, y_coord, marker='o', color='blue', markersize=10)
                
                plt.show()
                
                pprint.pprint(data['liveData']['plays']['allPlays'][event_idx])
            except:
                pass

interactive(children=(IntSlider(value=2016, description='Year', max=2020, min=2016), Dropdown(description='Sea…